In [5]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
output_notebook()

def get_product_agg(cols):
    # type: (object) -> object
    df_train = pd.read_csv('../input/train_sample_100.csv', usecols = ['Semana', 'Producto_ID'] + cols,
                           dtype  = {'Semana': 'int32',
                                     'Producto_ID':'int32',
                                     'Venta_hoy':'float32',
                                     'Venta_uni_hoy': 'int32',
                                     'Dev_uni_proxima':'int32',
                                     'Dev_proxima':'float32',
                                     'Demanda_uni_equil':'int32'})
    agg  = df_train.groupby(['Semana', 'Producto_ID'], as_index=False).agg(['count','sum', 'min', 'max','median','mean'])
    agg.columns  =  ['_'.join(col).strip() for col in agg.columns.values]
    del(df_train)
    return agg

Loading BokehJS ...

### a simple product aggregate. Kaggle computing power allows us to only calculate 1-2 fields at a time.

In [6]:
agg1 = get_product_agg(['Demanda_uni_equil','Dev_uni_proxima'])

In [7]:
agg1.shape

(90, 12)

In [8]:
agg1.head()

Dev_uni_proxima_count  Dev_uni_proxima_sum  \
Semana Producto_ID                                               
3      1064                             1                    0   
       1309                             1                    0   
       3631                             1                    0   
       4259                             2                    0   
       35452                            1                    0   

                    Dev_uni_proxima_min  Dev_uni_proxima_max  \
Semana Producto_ID                                             
3      1064                           0                    0   
       1309                           0                    0   
       3631                           0                    0   
       4259                           0                    0   
       35452                          0                    0   

                    Dev_uni_proxima_median  Dev_uni_proxima_mean  \
Semana Producto_ID                                                 
3      1064                              0                     0   
       1309                              0                     0   
       3631                              0                     0   
       4259                              0                     0   
       35452                             0                     0   

                    Demanda_uni_equil_count  Demanda_uni_equil_sum  \
Semana Producto_ID                                                   
3      1064                               1                      2   
       1309                               1                      5   
       3631                               1                      1   
       4259                               2                     10   
       35452                              1                      1   

                    Demanda_uni_equil_min  Demanda_uni_equil_max  \
Semana Producto_ID                                                 
3      1064                             2                      2   
       1309                             5                      5   
       3631                             1                      1   
       4259                             2                      8   
       35452                            1                      1   

                    Demanda_uni_equil_median  Demanda_uni_equil_mean  
Semana Producto_ID                                                    
3      1064                              2.0                     2.0  
       1309                              5.0                     5.0  
       3631                              1.0                     1.0  
       4259                              5.0                     5.0  
       35452                             1.0                     1.0

In [9]:
agg2 = get_product_agg(['Venta_uni_hoy'])
agg = agg1.join(agg2)

Let's preprocess products a little bit. I borrowed some of the preprocessing from [here](https://www.kaggle.com/lyytinen/grupo-bimbo-inventory-demand/basic-preprocessing-for-products) 

In [10]:
products  =  pd.read_csv("../input/producto_tabla.csv")
products['short_name'] = products.NombreProducto.str.extract('^(\D*)', expand=False)
products['brand'] = products.NombreProducto.str.extract('^.+\s(\D+) \d+$', expand=False)
w = products.NombreProducto.str.extract('(\d+)(Kg|g)', expand=True)
products['weight'] = w[0].astype('float')*w[1].map({'Kg':1000, 'g':1})
products['pieces'] =  products.NombreProducto.str.extract('(\d+)p ', expand=False).astype('float')
products.head()

,Producto_ID,NombreProducto,short_name,brand,weight,pieces
0,0,NO IDENTIFICADO 0,NO IDENTIFICADO,IDENTIFICADO,NaN,NaN
1,9,Capuccino Moka 750g NES 9,Capuccino Moka,NES,750.0,NaN
2,41,Bimbollos Ext sAjonjoli 6p 480g BIM 41,Bimbollos Ext sAjonjoli,BIM,480.0,6.0
3,53,Burritos Sincro 170g CU LON 53,Burritos Sincro,LON,170.0,NaN
4,72,Div Tira Mini Doradita 4p 45g TR 72,Div Tira Mini Doradita,TR,45.0,4.0


In [11]:
products.tail()

,Producto_ID,NombreProducto,short_name,brand,weight,pieces
2587,49992,Tostado Integral 180g MTA WON 49992,Tostado Integral,WON,180.0,NaN
2588,49993,Tostado Integral 180g TAB WON 49993,Tostado Integral,WON,180.0,NaN
2589,49994,Tostado Int 0pct Grasa Azuc 200g WON 49994,Tostado Int,WON,200.0,NaN
2590,49996,Tostado Int 0pct Grasa Azuc 200g MTA WON 49996,Tostado Int,WON,200.0,NaN
2591,49997,Tostado Int 0pct Grasa Azuc 200g TAB WON 49997,Tostado Int,WON,200.0,NaN


In [12]:
products.short_name.value_counts(dropna=False)

Pan Blanco                       39
Tortillinas                      37
Deliciosas Vainilla              35
Medias Noches                    25
Wonder                           22
Pan Integral                     19
Deliciosas Chochitos             17
Canelitas                        17
Suavicremas Chocolate            17
Suavicremas Vainilla             17
Choco Roles                      16
Suavicremas Fresa                16
Mantecadas Nuez                  15
Chocochispas                     15
Tostada Ondulada Tubo            15
Canapinas                        15
Tostado                          14
Barritas Fresa                   14
Barritas Pina                    13
Bran Frut Fresa                  12
Triki Trakes                     12
Deliciosas Chocochispas          12
Barra Multigrano Nuez            12
Mantecadas                       12
Totopo Chilaquiles               12
Plativolos                       12
Galleta Gansito                  12
Bollos BK                   

There are some weird products that weight 42 Kilos. Check out this Exhibidor :

![Exhibidor bimbo](https://mir-s3-cdn-cf.behance.net/project_modules/disp/55c94f24003843.5632c737c062c.jpeg)

In [13]:
sns.distplot(products.weight.dropna())

distribution of pieces

In [14]:
sns.distplot(products.pieces.dropna())

#### Lets clean up product names a bit, we have ~1000 unique names once we cleaned the weights, but there is much more work to be done
#### Products have some abbreviation leftovers that I did not clean, products have similar names but different word forms, etc.

In [15]:
products.short_name.nunique()

1014

Let's clean stop words and leave only the word stems (I did not clean abbreviations, sorry)

In [16]:
from nltk.corpus import stopwords
print(stopwords.words("spanish"))

[u'de', u'la', u'que', u'el', u'en', u'y', u'a', u'los', u'del', u'se', u'las', u'por', u'un', u'para', u'con', u'no', u'una', u'su', u'al', u'lo', u'como', u'm\xe1s', u'pero', u'sus', u'le', u'ya', u'o', u'este', u's\xed', u'porque', u'esta', u'entre', u'cuando', u'muy', u'sin', u'sobre', u'tambi\xe9n', u'me', u'hasta', u'hay', u'donde', u'quien', u'desde', u'todo', u'nos', u'durante', u'todos', u'uno', u'les', u'ni', u'contra', u'otros', u'ese', u'eso', u'ante', u'ellos', u'e', u'esto', u'm\xed', u'antes', u'algunos', u'qu\xe9', u'unos', u'yo', u'otro', u'otras', u'otra', u'\xe9l', u'tanto', u'esa', u'estos', u'mucho', u'quienes', u'nada', u'muchos', u'cual', u'poco', u'ella', u'estar', u'estas', u'algunas', u'algo', u'nosotros', u'mi', u'mis', u't\xfa', u'te', u'ti', u'tu', u'tus', u'ellas', u'nosotras', u'vosostros', u'vosostras', u'os', u'm\xedo', u'm\xeda', u'm\xedos', u'm\xedas', u'tuyo', u'tuya', u'tuyos', u'tuyas', u'suyo', u'suya', u'suyos', u'suyas', u'nuestro', u'nuestra', 

In [17]:
products['short_name_processed'] = (products['short_name']
                                        .map(lambda x: " ".join([i for i in x.lower()
                                                                 .split() if i not in stopwords.words("spanish")])))

In [18]:
products['short_name_processed'].nunique()

992

In [19]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("spanish")

In [20]:
print(stemmer.stem("Tortillas"))

tortill


In [21]:
products['short_name_processed'] = (products['short_name_processed']
                                        .map(lambda x: " ".join([stemmer.stem(i) for i in x.lower().split()])))

In [22]:
products.short_name_processed.nunique()

963

In [23]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 1000) 

product_bag_words = vectorizer.fit_transform(products.short_name_processed).toarray()
product_bag_words.shape

(2592, 688)

In [24]:
vectorizer.get_feature_names()

[u'abc',
 u'abej',
 u'aceit',
 u'actifresh',
 u'agu',
 u'ajn',
 u'ajon',
 u'ajonj',
 u'ajonjoli',
 u'alka',
 u'almendr',
 u'almenuez',
 u'almoh',
 u'amarant',
 u'american',
 u'ami',
 u'and',
 u'angus',
 u'animalit',
 u'apapach',
 u'apar',
 u'arandalet',
 u'arandan',
 u'arroz',
 u'asad',
 u'atun',
 u'avellanafs',
 u'aven',
 u'azuc',
 u'azul',
 u'ba',
 u'bagel',
 u'bagels',
 u'baguett',
 u'bairds',
 u'baji',
 u'balcon',
 u'banderill',
 u'bar',
 u'barbaco',
 u'barr',
 u'barri',
 u'barrit',
 u'bas',
 u'baul',
 u'bc',
 u'bco',
 u'bear',
 u'bes',
 u'besnuez',
 u'best',
 u'betun',
 u'bicolor',
 u'big',
 u'bigot',
 u'bim',
 u'bimb',
 u'bimbocontig',
 u'bimboll',
 u'bimbolunch',
 u'bimboton',
 u'bimbunuel',
 u'bimking',
 u'bisabor',
 u'biscotel',
 u'bisquet',
 u'bk',
 u'blanc',
 u'blist',
 u'bolill',
 u'boll',
 u'bols',
 u'bolsit',
 u'bombonet',
 u'bon',
 u'bran',
 u'branfrut',
 u'brav',
 u'bre',
 u'briobalanc',
 u'brioch',
 u'britt',
 u'bubulubu',
 u'burrer',
 u'burrit',
 u'butt',
 u'buttermil

In [25]:
product_bag_words = pd.concat([products.Producto_ID, 
                               pd.DataFrame(product_bag_words, 
                                            columns= vectorizer.get_feature_names(), index = products.index)], axis=1)
product_bag_words.head()

,Producto_ID,abc,abej,aceit,actifresh,agu,ajn,ajon,ajonj,ajonjoli,...,wb,wendys,whit,whol,wond,wonderbutt,yerbabuen,you,zarzamor,zer
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,41,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,53,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,72,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
product_bag_words.drop('Producto_ID', axis=1).sum().sort_values(ascending=False).head(100)

pan          220
fres         157
tost         115
duo          101
tub          100
vainill       94
delici        92
blanc         91
chocolat      91
gallet        84
gansit        80
barrit        77
tortill       76
suavicrem     69
nuez          63
multigran     63
frut          63
pin           61
lat           58
bran          57
mantec        56
boll          56
tortillin     55
princip       55
triki         51
mini          51
kc            50
integral      50
choc          49
medi          48
            ... 
mm            20
jamon         20
extra         19
azuc          19
pinguin       19
burrit        18
conch         18
chochit       18
ric           18
rock          18
bk            18
cuernit       17
canapin       17
reban         17
tir           16
bigot         16
panqu         16
escol         15
salv          15
sponch        15
fs            15
mari          15
chisp         15
bols          15
thins         14
dalmat        14
orejit        14
pastiset      

####  let's have a look, what is the product with the biggest demand of all times

In [27]:
df = (pd.merge(agg1.reset_index(), products, on='Producto_ID', how='left').
      groupby('short_name')['Demanda_uni_equil_sum'].sum().sort_values(ascending=False))

In [28]:
plt.figure(figsize = (12,15))
df.head(50).sort_values().plot(kind='barh')

The best seller is by far Nito. Check this out, looks yummy : 

![](http://static.manufactura.mx/media/2013/11/20/negrito.jpg)

#### a quick look at the distributions

In [29]:
sns.distplot(df)

In [30]:
sns.distplot(np.log1p(df))

#### expanding the aggregate

In [31]:
df = (pd.merge(agg.reset_index(), products, on='Producto_ID', how='left').
      groupby('short_name')['Demanda_uni_equil_sum', 'Venta_uni_hoy_sum', 'Dev_uni_proxima_sum', 'Dev_uni_proxima_count']
      .sum().sort_values(by = 'Demanda_uni_equil_sum', ascending=False))

In [32]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Demanda_uni_equil_sum,52.0,10.692308,15.126316,1.0,3.00,6.0,10.5,96.0
Venta_uni_hoy_sum,52.0,10.846154,15.084578,1.0,3.75,6.5,10.5,96.0
Dev_uni_proxima_sum,52.0,0.173077,0.785185,0.0,0.00,0.0,0.0,4.0
Dev_uni_proxima_count,52.0,1.923077,1.265626,1.0,1.00,1.5,2.0,6.0


There are interesting things. There are products for which Demanda_uni_equil_sum = 0 and other fields are not equal to 0

In [33]:
df[df.Demanda_uni_equil_sum == 0].count()

Demanda_uni_equil_sum    0
Venta_uni_hoy_sum        0
Dev_uni_proxima_sum      0
Dev_uni_proxima_count    0
dtype: int64

In [34]:
df[df.Demanda_uni_equil_sum == 0]

,Demanda_uni_equil_sum,Venta_uni_hoy_sum,Dev_uni_proxima_sum,Dev_uni_proxima_count
short_name,,,,


Similarly there are products with 0 sales and only returns

In [35]:
df[df.Venta_uni_hoy_sum == 0]

,Demanda_uni_equil_sum,Venta_uni_hoy_sum,Dev_uni_proxima_sum,Dev_uni_proxima_count
short_name,,,,


there are products that were never returned

In [36]:
df[df.Dev_uni_proxima_sum == 0].count()

Demanda_uni_equil_sum    49
Venta_uni_hoy_sum        49
Dev_uni_proxima_sum      49
Dev_uni_proxima_count    49
dtype: int64

In [37]:
df[df.Dev_uni_proxima_sum == 0].head(20)

,Demanda_uni_equil_sum,Venta_uni_hoy_sum,Dev_uni_proxima_sum,Dev_uni_proxima_count
short_name,,,,
Nito,96,96,0,5
Pinguinos,50,50,0,4
Rebanada Sandwich MP,30,30,0,1
Pan Integral,29,29,0,6
Dalmata,25,25,0,5
Leche Nito,21,21,0,2
Bimbunuelos,19,19,0,4
Rebanada,18,18,0,2
Tostado,17,17,0,2


Let's cut products into 10 quantiles by summary adjusted demand.

In [38]:
df['Q'] = pd.qcut(df.Demanda_uni_equil_sum, 10)
df.Q.value_counts()

ValueError: Bin edges must be unique: array([  1. ,   2. ,   3. ,   4. ,   4. ,   6. ,   8. ,   9.7,  14.8,
        20.8,  96. ])

In [ ]:
df[df.Q == '[0, 49]'].index.values

distribution of returns by product

In [ ]:
sns.distplot(df.Dev_uni_proxima_sum)

In [ ]:
sns.distplot(np.log1p(df.Dev_uni_proxima_sum))

Distribution of counts by product

In [ ]:
sns.distplot(df.Dev_uni_proxima_count)

In [ ]:
sns.distplot(np.log1p(df.Dev_uni_proxima_count))

#### lets aggregate by week and short_name now

In [ ]:
df_hmp = (pd.merge(agg.reset_index(), products, on='Producto_ID', how='left').
      groupby(['Semana','short_name'])['Demanda_uni_equil_sum', 'Venta_uni_hoy_sum', 'Dev_uni_proxima_sum', 'Dev_uni_proxima_count'].sum().reset_index())

In [ ]:
df_hmp.head()

#### a quick check if demand distribution changes week to week

In [ ]:
df_hmp['log1p_Demanda_uni_equil_sum'] = np.log1p(df_hmp.Demanda_uni_equil_sum)
g = sns.FacetGrid(df_hmp, row = 'Semana')
g = g.map(sns.distplot, 'log1p_Demanda_uni_equil_sum')

#### Now let's look at which proucts sell by week with interactive heatmaps. Let's use our quantiles here.

In [ ]:
from bokeh.charts import HeatMap
from bokeh.plotting import vplot

heatmaps = []
for i in df.Q.cat.categories.values:
    hm = HeatMap(df_hmp[df_hmp.short_name.isin(df[df.Q == i].index.values)],
                        x='short_name', y = 'Semana', values = 'Demanda_uni_equil_sum',
                 hover_tool = True, title = 'Products with summary demand '+ str(i), xgrid = False,
                 stat = 'sum',plot_width=950, plot_height=400, tools='hover, box_zoom, resize, save, wheel_zoom, reset',
                 )
    heatmaps.append(hm)
show(vplot(*heatmaps))

#### Same series of charts but for returns

In [ ]:
from bokeh.charts import HeatMap
from bokeh.plotting import vplot
df['Q_ret'] = pd.qcut(df.Dev_uni_proxima_sum, 5)
heatmaps = []
for i in df.Q_ret.cat.categories.values:
    hm = HeatMap(df_hmp[df_hmp.short_name.isin(df[df.Q_ret == i].index.values)],
                        x='short_name', y = 'Semana', values = 'Demanda_uni_equil_sum',
                 hover_tool = True, title = 'Products with summary returns '+ str(i), xgrid = False,
                 stat = 'sum',plot_width=800, plot_height=400, tools='hover, box_zoom, resize, save, wheel_zoom, reset',
                 )
    heatmaps.append(hm)
show(vplot(*heatmaps))